In [1]:
import numpy as np
import gzip

f = gzip.open('images.gz','r')

image_size = 28
num_images = 100

f.read(16)
buf = f.read(image_size * image_size * num_images)
data = np.frombuffer(buf, dtype=np.uint8).astype(np.float32)
data = data.reshape(num_images, image_size, image_size, 1)

In [2]:
def to_one_list(number):
    t = []    
    for row in number:
        for pixel in row:
            t.append(pixel)
    return t

In [3]:
new_data = []

for number in data:
    number = to_one_list(number.squeeze())
    new_data.append(number)

In [4]:
def grayscale(img):
    new_img = []
    
    for pixel in img:
        if pixel > (255 / 2):
            pixel = 1
        else:
            pixel = 0
        
        new_img.append(pixel)

    return new_img

In [5]:
for i in range(len(new_data)):
    new_data[i] = grayscale(new_data[i])

In [6]:
f = gzip.open('labels.gz','r')
new_labels = []

for i in range(0,100):
    f.read(8)
    buf = f.read(1 * 32)
    labels = np.frombuffer(buf, dtype=np.uint8).astype(np.int64)
    
    for label in labels:
        new_labels.append([label])

In [7]:
def sigmoid(x):
    return 1.0/(1+ np.exp(-x))

def sigmoid_derivative(x):
    return x * (1.0 - x)

class NeuralNetwork:
    def __init__(self, x, y):
        self.input = x
        self.weights1 = np.random.rand(self.input.shape[1],5) 
        self.weights2 = np.random.rand(5,1)                 
        self.y = y
        self.output = np.zeros(self.y.shape)

    def feedforward(self):
        self.layer1 = sigmoid(np.dot(self.input, self.weights1))
        self.output = sigmoid(np.dot(self.layer1, self.weights2))

    def backprop(self):
        d_weights2 = np.dot(self.layer1.T, (2*(self.y - self.output) * sigmoid_derivative(self.output)))
        d_weights1 = np.dot(self.input.T,  (np.dot(2*(self.y - self.output) * sigmoid_derivative(self.output), self.weights2.T) * sigmoid_derivative(self.layer1)))

        self.weights1 += d_weights1
        self.weights2 += d_weights2

In [17]:
X = np.array(list(new_data[:5])) / 10
y = np.array(new_labels[:5]) / 10
nn = NeuralNetwork(X,y)

for i in range(10000):
    nn.feedforward()
    nn.backprop()

print(nn.output * 10)

[[4.99979012]
 [0.06070114]
 [3.99977108]
 [0.99915825]
 [8.99791801]]


In [9]:
new_labels[:5]

[[5], [0], [4], [1], [9]]